In [ ]:
#!/usr/bin/env python3
"""
Download daily tomato mandi prices and meteorological data (2020–2025).
"""

import os
import requests

# %% [1] Configuration & API Keys
DATA_GOV_KEY = os.getenv("DATA_GOV_KEY")
if not DATA_GOV_KEY:
    raise ValueError("Set the DATA_GOV_KEY environment variable")  #

# AGMARKNET resource ID for daily mandi prices of various commodities
RESOURCE_ID = "9ef84268-d588-465a-a308-a864a43d0070"  # :contentReference[oaicite:0]{index=0}

# NASA POWER endpoint for daily point queries
NASA_POWER_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"  # :contentReference[oaicite:1]{index=1}

# %% [2] Download Tomato Prices from Data.gov.in AGMARKNET API
def download_tomato_prices(output_path="tomato_prices_daily.csv"):
    """
    Fetches daily mandi prices for Tomato from Data.gov.in API and saves to CSV.
    """
    params = {
        "api-key": DATA_GOV_KEY,
        "resource_id": RESOURCE_ID,
        "format": "csv",
        "filters[commodity]": "Tomato",
        "filters[state]": "",        # leave blank or specify if you want a particular state
        "filters[district]": "",     # leave blank or specify a district
        "limit": 50000,              # adjust as needed
        "offset": 0
    }
    print("Requesting tomato prices...")
    r = requests.get("https://api.data.gov.in/resource/" + RESOURCE_ID, params=params, timeout=30)
    r.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(r.content)
    print(f"Saved tomato prices to {output_path}")  # :contentReference[oaicite:2]{index=2}

# %% [3] Download Meteorological Data from NASA POWER
def download_nasa_power(lat, lon, out_csv="nasa_power_daily.csv"):
    """
    Fetches daily T2M (temperature) and PRECTOT (precipitation) for given lat/lon.
    """
    # Define parameters to pull
    payload = {
        "parameters": "T2M,PRECTOT",
        "community":  "AG",
        "latitude":   lat,
        "longitude":  lon,
        "start":      "20200101",
        "end":        "20251231",
        "format":     "CSV"
    }
    print(f"Requesting NASA POWER data for ({lat}, {lon})...")
    r = requests.get(NASA_POWER_URL, params=payload, timeout=60)
    r.raise_for_status()
    with open(out_csv, "wb") as f:
        f.write(r.content)
    print(f"Saved NASA POWER data to {out_csv}")  # :contentReference[oaicite:3]{index=3}

# %% [4] Main Execution
if __name__ == "__main__":
    # 1) Download Tomato Prices
    download_tomato_prices("tomato_prices_daily.csv")

    # 2) Download NASA POWER for a representative point in India
    #    (e.g., Madhya Pradesh central coordinates; adjust as needed)
    download_nasa_power(lat=23.2599, lon=77.4126, out_csv="nasa_power_daily.csv")
